In [ ]:
from parser.models.question import (
    Question,
    SubQuestion,
    SubSubQuestion,
    MultipleChoiceQuestion,
)
from parser.models.syllabus import Sy
















from typing import List, Optional
import re
import tqdm
import pickle
import os
import numpy as np
from collections import Counter
from sentence_transformers import SentenceTransformer

c:\Users\wzhzhang\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[WARNING] From c:\Users\wzhzhang\AppData\Local\Programs\Python\Python312\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

[INFO] PyTorch version 2.5.1+cu121 available.
[INFO] TensorFlow version 2.19.0 available.


In [ ]:
class Classifier:

    def __init__(
        self,
        syllabuses: List[Syllabus],
        batch_size: int = 64,
        cache_path: str = "syllabus_embeddings.pkl",
        model_name: str = "sentence-transformers/all-mpnet-base-v2",  # Smaller general-purpose model
    ):
        self.batch_size = batch_size
        self.cache_path = cache_path
        self.syllabus_objects = syllabuses

        print(f"Initializing SentenceTransformer with model: {model_name}")
        self.model = SentenceTransformer(model_name)

        # Try to load cache, otherwise process syllabuses
        if os.path.exists(cache_path) and self._try_load_cache():
            print(f"Loaded embeddings from cache: {cache_path}")
        else:
            print("Processing syllabus content...")
            self._preprocess_syllabuses(syllabuses)
            self._save_cache()

    def _preprocess_syllabuses(self, syllabuses: List[Syllabus]) -> None:
        # Create corpus and mapping
        self.corpus = []
        self.syllabus_mapping = []

        for idx, syllabus in enumerate(syllabuses):
            for point in syllabus.content:
                text = point.lower().strip()
                self.corpus.append(text)
                self.syllabus_mapping.append(idx)

        print(f"Encoding {len(self.corpus)} syllabus points")
        self.corpus_embeddings = self.model.encode(
            self.corpus,
            batch_size=self.batch_size,
            show_progress_bar=True,
            convert_to_numpy=True,
        )

    def _save_cache(self) -> None:
        cache_data = {
            "corpus": self.corpus,
            "corpus_embeddings": self.corpus_embeddings,
            "syllabus_mapping": self.syllabus_mapping,
        }
        with open(self.cache_path, "wb") as f:
            pickle.dump(cache_data, f)
        print(f"Embeddings cached to {self.cache_path}")

    def _try_load_cache(self) -> bool:
        try:
            with open(self.cache_path, "rb") as f:
                cache_data = pickle.load(f)
                self.corpus = cache_data["corpus"]
                self.corpus_embeddings = cache_data["corpus_embeddings"]
                self.syllabus_mapping = cache_data["syllabus_mapping"]
            return True
        except Exception as e:
            print(f"Cache load failed: {str(e)}")
            return False

    def classify_all(
        self, questions: List[Question | MultipleChoiceQuestion]
    ) -> List[Question | MultipleChoiceQuestion]:
        for question in tqdm.tqdm(questions, desc="Classifying questions"):
            self.classify(question)
        return questions

    def classify(
        self, question: MultipleChoiceQuestion | Question | SubQuestion | SubSubQuestion
    ) -> None:
        if isinstance(question, MultipleChoiceQuestion):
            question.syllabus = self.get_best_syllabus(question.text)
            return
        elif isinstance(question, SubQuestion):
            if question.subsubquestions:
                for subsubquestion in question.subsubquestions:
                    self.classify(subsubquestion)
        elif isinstance(question, Question):
            if question.subquestions:
                for subquestion in question.subquestions:
                    self.classify(subquestion)

        # Combine question text and answer for better matching
        question_text = (
            question.text + " " + (question.answer if question.answer else "")
        )
        question.syllabus = self.get_best_syllabus(question_text)

    def get_best_syllabus(self, question_sentence: str, threshold = 0.5) -> Syllabus:
        # Clean text: remove more than three connected dots and score markers
        question_sentence = re.sub(r"\.{3,}", "", question_sentence)
        question_sentence = re.sub(r"\[\d+\]", "", question_sentence)
        question_sentence = re.sub(r"\(\w{1,3}\)", "", question_sentence)

        # Get embedding for the question
        question_embedding = self.model.encode(question_sentence, convert_to_numpy=True)

        # Calculate cosine similarities
        similarities = np.dot(self.corpus_embeddings, question_embedding) / (
            np.linalg.norm(self.corpus_embeddings, axis=1)
            * np.linalg.norm(question_embedding)
        )

        # Get top 10 most similar syllabus points
        top_indices = np.argsort(similarities)[-10:][::-1]

        # Count syllabus occurrences to find the most likely match
        syllabus_votes = Counter()

        for idx in top_indices:
            syllabus_idx = self.syllabus_mapping[idx]
            score = similarities[idx]
            if score > threshold: 
                syllabus_votes[syllabus_idx] += score

        # If we got any valid results, return the most common syllabus
        if syllabus_votes:
            best_syllabus_idx = syllabus_votes.most_common(1)[0][0]
            return self.syllabus_objects[best_syllabus_idx]

        # Fallback if no matches
        return Syllabus("0", "Unknown")

In [3]:
with pdfplumber.open("papers/595426-2023-2025-syllabus.pdf") as syllabus_pdf:
        syllabus_parser = SyllabusParser(syllabus_pdf, pages=(12, 46))
        syllabuses = syllabus_parser.parse_syllabus()
with pdfplumber.open("papers/igcse-biology-0610/0610_w22_qp_42.pdf") as qppdf:
    sq_parser = QuestionPaperParser(qppdf, image_prefix="0610_w22_qp_42")
    questions = sq_parser.parse_question_paper()
sqms_parser = SQMSParser("papers/igcse-biology-0610/0610_w22_ms_42.pdf", questions)
questions = sqms_parser.parse_ms()

49.6063


[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[WARNING] Ignore Line "<image>" due to overlap
[INFO] [3/4] Parsing pages...
[INFO] (1/15) Page 1
[INFO] (2/15) Page 2
[INFO] (3/15) Page 3
[INFO] (4/15) Page 4
[INFO] (5/15) Page 5
[INFO] (6/15) Page 6
[INFO] (7/15) Page 7
[INFO] (8/15) Page 8
[INFO] (9/15) Page 9
[INFO] (10/15) Page 10
[INFO] (11/15) Page 11
[INFO] (12/15) Page 12
[INFO] (13/15) Page 13
[INFO] (14/15) Page 14
[INFO] (15/15) Page 15


In [4]:
classifier = Classifier(syllabuses, cache_path="biology_syllabus.pt")
questions = classifier.classify_all(questions)

Initializing SentenceTransformer with model: sentence-transformers/all-mpnet-base-v2


[INFO] Use pytorch device_name: cuda:0
[INFO] Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Loaded embeddings from cache: biology_syllabus.pt


Classifying questions: 100%|██████████| 6/6 [00:03<00:00,  1.53it/s]


In [5]:
def format_question_hierarchy(questions):
    output = ""
    for q in questions:
        output += f"{q.text}\n{q.syllabus.title if hasattr(q, "syllabus") else ""}\n "
        if q.subquestions:
            for sub_q in q.subquestions:
                text = sub_q.text.strip()
                output += f"\n    {text}\n{sub_q.syllabus.title if hasattr(sub_q, "syllabus") else ""}"
                if sub_q.subsubquestions:
                    for subsub_q in sub_q.subsubquestions:
                        text = subsub_q.text.strip()
                        subsub_q: SubSubQuestion
                        output += f"\n        {text}\n{subsub_q.syllabus.title if hasattr(subsub_q, "syllabus") else ""}\n"
        output += "\n" + "-" * 80 + "\n"
    return output.strip()
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(format_question_hierarchy(questions))